In [1]:
import os
data_file_dir="/home/avnish/iNeuron_Private_Intelligence_Limited/industry_ready_project/live_class/sensor-fault-detection/artifact/10_29_2022_12_29_11/data_ingestion/ingested"

In [2]:

train_file_path,test_file_path = [os.path.join(data_file_dir,files) for files in os.listdir(data_file_dir)]

In [3]:
import tensorflow_data_validation as tfdv

2022-10-29 22:53:48.786533: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-29 22:53:48.991004: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-29 22:53:48.996476: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-29 22:53:48.996510: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [5]:
train_stats = tfdv.generate_statistics_from_csv(data_location=train_file_path)

In [7]:
tfdv.visualize_statistics(train_stats)

In [8]:
schema = tfdv.infer_schema(train_stats)

In [11]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'class',STRING,required,,'class'
'aa_000',INT,required,,-
'ab_000',INT,optional,single,-
'ac_000',INT,optional,single,-
'ad_000',INT,optional,single,-
...,...,...,...,...
'ee_007',INT,optional,single,-
'ee_008',INT,optional,single,-
'ee_009',INT,optional,single,-


,Values
Domain,
'class',"'neg', 'pos'"


In [12]:
test_stats = tfdv.generate_statistics_from_csv(test_file_path)

In [15]:
anomalies = tfdv.validate_statistics(statistics=test_stats,schema=schema)

In [17]:
tfdv.display_anomalies(anomalies=anomalies)

In [48]:
# tfdv.get_feature(schema, 'aa_000').skew_comparator.infinity_norm.threshold = 0.01
tfdv.get_feature(schema, 'aa_000').skew_comparator.jensen_shannon_divergence.threshold = 0.01
# jensen_shannon_divergence

In [49]:
skew_anomalies = tfdv.validate_statistics(
        statistics=train_stats, schema=schema,serving_statistics=test_stats)

In [50]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'class',STRING,required,,'class'
'aa_000',INT,required,,-
'ab_000',INT,optional,single,-
'ac_000',INT,optional,single,-
'ad_000',INT,optional,single,-
...,...,...,...,...
'ee_007',INT,optional,single,-
'ee_008',INT,optional,single,-
'ee_009',INT,optional,single,-


,Values
Domain,
'class',"'neg', 'pos'"


In [51]:
tfdv.display_anomalies(skew_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'aa_000',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.107506 (up to six significant digits), above the threshold 0.01."


In [55]:
    # Assume we have already generated the statistics of training dataset for
    # day 2, and inferred a schema from it.
    # train_day1_stats = tfdv.generate_statistics_from_tfrecord(data_location=T)
    # Add a drift comparator to schema for 'payment_type' and set the threshold
    # of L-infinity norm for triggering drift anomaly to be 0.01.
tfdv.get_feature(schema, 'aa_000').drift_comparator.infinity_norm.threshold = 0.01
drift_anomalies = tfdv.validate_statistics(
        statistics=train_stats, schema=schema, previous_statistics=test_stats)

In [56]:
tfdv.display_anomalies(drift_anomalies)

In [78]:
with open("schema.pb","wb") as file:
    file.write(schema.SerializeToString())

In [79]:
type(schema)

tensorflow_metadata.proto.v0.schema_pb2.Schema

In [ ]:
from tensorflow_metadata.proto.v0.schema_pb2 import Schema

In [91]:
from google.protobuf import text_format
from tensorflow.python.lib.io import file_io
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow_metadata.proto.v0.anomalies_pb2 import Anomalies

def write_schema(schema, output_path):
  schema_text = text_format.MessageToString(schema)
  file_io.write_string_to_file(output_path, schema_text)

def load_schema(input_path):
  schema = Anomalies()#schema_pb2.Schema()
  schema_text = file_io.read_file_to_string(input_path)
  text_format.Parse(schema_text, schema)
  return schema      

In [90]:
from tensorflow_metadata.proto.v0.anomalies_pb2 import Anomalies

In [88]:
type(anomalies)

tensorflow_metadata.proto.v0.anomalies_pb2.Anomalies

In [86]:
write_schema(anomalies,"schema.pb")

In [93]:
tfdv.display_anomalies(load_schema("schema.pb"))